# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Creating-XML-for-NETCONF-via-LXML" data-toc-modified-id="Creating-XML-for-NETCONF-via-LXML-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creating XML for NETCONF via LXML</a></div><div class="lev1 toc-item"><a href="#Alternative-One:-Namespaces" data-toc-modified-id="Alternative-One:-Namespaces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Alternative One: Namespaces</a></div><div class="lev2 toc-item"><a href="#Namespaces-we-care-about" data-toc-modified-id="Namespaces-we-care-about-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Namespaces we care about</a></div><div class="lev2 toc-item"><a href="#Register-the-namespaces-with-ElementTree" data-toc-modified-id="Register-the-namespaces-with-ElementTree-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Register the namespaces with ElementTree</a></div><div class="lev3 toc-item"><a href="#Using-xpath-to-retrieve-Elements" data-toc-modified-id="Using-xpath-to-retrieve-Elements-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Using xpath to retrieve Elements</a></div><div class="lev1 toc-item"><a href="#Alternative-Two:-No-(Less)-Namespaces" data-toc-modified-id="Alternative-Two:-No-(Less)-Namespaces-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Alternative Two: No (Less) Namespaces</a></div><div class="lev1 toc-item"><a href="#Writing-XML-to-a-File" data-toc-modified-id="Writing-XML-to-a-File-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Writing XML to a File</a></div>

# Creating XML for NETCONF via LXML
LXML is a Python library to deal with XML element trees (among other things). It provides quite a lot of capabilities and in the context of NETCONF here's a little sample code that shows some ways how to create XML data for NETCONF via a script (vs. manually copyingand pasting it). 

But first of all let's lay some ground work:

In [ ]:
from lxml import etree as ET

In [ ]:
def pp(element):
    print(ET.tostring(element, pretty_print=True).decode('utf-8'))

# Alternative One: Namespaces
This approach actually uses namespaces throughout the creation of the objects. This might look a bit over the top but it actually helps a lot to understand where those namespaces come into play. When looking at e.g. the `ietf-interfaces.yang` model, we can see pretty much at the top:

    module ietf-interfaces {

      namespace "urn:ietf:params:xml:ns:yang:ietf-interfaces";
      prefix if;

These statements define the namespace and the prefix for the given namespace. We will see these two elements later in our XML creation for an interface element in the `ietf-interfaces` YANG model.

## Namespaces we care about
The following three namespaces are relevant int the context of `ietf-interfaces` YANG model or more specifically to describe an interface within that model.

In [ ]:
if_ns = ('if', 'urn:ietf:params:xml:ns:yang:ietf-interfaces')
it_ns = ('ianaift', 'urn:ietf:params:xml:ns:yang:iana-if-type')
ip_ns = ('ip', 'urn:ietf:params:xml:ns:yang:ietf-ip')

## Register the namespaces with ElementTree
If we register the namespace with ET automatically creates the proper prefix ('if' for ietf-interfaces) when creating an element without explicitely specifying a namespace map IF the element tag is properly specified **with** a namespace designator. This looks like

    {urn:ietf:params:xml:ns:yang:ietf-interfaces}interfaces
    
The below is a bit of a hack to coerce the namespace tuples we defined above into function paramenters. Actually, `register_namespace()` takes two: 

1. the prefix -and- 
2. the actual URN for the namespace. 

In [ ]:
ET.register_namespace(*if_ns)
ET.register_namespace(*it_ns)
ET.register_namespace(*ip_ns)

Now, let's create a 'config' container with several sub-containers which eventually make up an ietf-interfaces interface tree describing a loopback interface with a name, a state and some IP addresses.

In [ ]:
root = ET.Element('config')

# Interface container
interfaces = ET.SubElement(root, '{%s}interfaces' % if_ns[1])
interface = ET.SubElement(interfaces, '{%s}interface' % if_ns[1])

# Interface Name
e = ET.SubElement(interface, '{%s}name' % if_ns[1])
e.text = 'Loopback99'

# enabled?
e = ET.SubElement(interface, '{%s}enabled' % if_ns[1])
e.text = 'true'

# Interface Type
e = ET.SubElement(interface, '{%s}type' % if_ns[1], nsmap=dict((if_ns, it_ns)))
e.text = '%s:softwareLoopback' % it_ns[0]

# IPv4 Element
ip = ET.SubElement(interface, '{%s}ipv4' % ip_ns[1])
address = ET.SubElement(ip, '{%s}address' % ip_ns[1])
e = ET.SubElement(address, '{%s}ip' % ip_ns[1])
e.text = '1.2.3.4'
e = ET.SubElement(address, '{%s}netmask' % ip_ns[1])
e.text = '255.0.0.0'

# IPv6 Element
ip = ET.SubElement(interface, '{%s}ipv6' % ip_ns[1])
address = ET.SubElement(ip, '{%s}address' % ip_ns[1])
e = ET.SubElement(address, '{%s}ip' % ip_ns[1])
e.text = '2001:db8:99::1'
e = ET.SubElement(address, '{%s}prefix-length' % ip_ns[1])
e.text = '64'

Finally, let's print the resulting ElementTree:

In [ ]:
pp(root)

As you can see, all of the elements within the `<config>` container are properly prefixed with the namespace and the proper `xmlns` attribute.

### Using xpath to retrieve Elements
We can use the `xpath` method to extract information from the ElementTree. Note that providing the proper namespace maps **and** prefixing the elements with their associated namespaces is required. Results are returned as a list.

In [ ]:
root.xpath('/config/if:interfaces/if:interface/ip:ipv6/ip:address/ip:ip/text()', namespaces=dict((if_ns, ip_ns)))

In [ ]:
pp(root.xpath('//*[local-name() = "ipv4"]')[0])

If we want to specify a particular element in the tree without specifying namespaces then we have to use the `local-name()` function:

In [ ]:
root.xpath('//*[local-name() = "ipv6"]/*[local-name()="address"]/*[local-name()="ip"]/text()')

In [ ]:
root.xpath('//*[local-name() = "ipv6/address/ip"]/text()')

In [ ]:
root.xpath('//*[local-name() = "ip"]/text()')

`xpath` is a powerful way to retrieve information from an XML tree. It can use expression functions, variable replacesment, regular expression and more. See [the documentation page](http://lxml.de/xpathxslt.html#xpath) for more examples and information. And some additional [examples on xpath functions](https://msdn.microsoft.com/en-us/library/ms256086&#40;v=vs.110&#41;.aspx).

# Alternative Two: No (Less) Namespaces
Or at least minimize the use of namespaces and operate primarily with the default namesspace. The default namespace has no prefix but elements can still carry the xmlns attributes to distinguish between different namespaces, where appropriate.

In [ ]:
if_ns = {None: 'urn:ietf:params:xml:ns:yang:ietf-interfaces'}
it_ns = {'ianaift': 'urn:ietf:params:xml:ns:yang:iana-if-type'}
ip_ns = {None: 'urn:ietf:params:xml:ns:yang:ietf-ip'}

In [ ]:
# root element
root = ET.Element('config')

# Interface container
interfaces = ET.SubElement(root, 'interfaces', nsmap=if_ns)
interface = ET.SubElement(interfaces, 'interface')

# Interface Name
e = ET.SubElement(interface, 'name')
e.text = 'Loopback99'

# enabled?
e = ET.SubElement(interface, 'enabled')
e.text = 'true'

# Interface Type
e = ET.SubElement(interface, 'type', nsmap={**if_ns, **it_ns})
e.text = 'ianaift:softwareLoopback'

# IPv4 Element
ip = ET.SubElement(interface, 'ipv4', nsmap=ip_ns)
address = ET.SubElement(ip, 'address')
e = ET.SubElement(address, 'ip')
e.text = '1.2.3.4'
e = ET.SubElement(address, 'netmask')
e.text = '255.0.0.0'

# IPv6 Element
ip = ET.SubElement(interface, 'ipv6', nsmap=ip_ns)
address = ET.SubElement(ip, 'address')
e = ET.SubElement(address, 'ip')
e.text = '2001:db8:99::1'
e = ET.SubElement(address, 'prefix-length')
e.text = '64'

In [ ]:
pp(root)

The resulting XML above looks a bit more concise then the previous result. Syntactically and also from a functionality point of view (in the ncclient / NETCONF context) they are exactly the same. It's up to the reader which method they prefer.

The idea here is to only pass a namespace map (`nsmap`) when required, e.g. when defining the first element within the tree with a different namespace than the previous element used.

There is one oddity in the above code worth mentioning: The construct `nsmap={**if_ns, **it_ns}`. What does this do? This is one way of 'merging' two dictionaries into one. Essentially `nsmap = if_ns + if_ns`. But this isn't defined in Python.

The reason why we need two namespaces for the `<type>` element is that the element itself is within the previous namespace ('if'). The value of the element, however, is from a different namespace ('ianaift'). This needs to be designated and hence the dual-namespace map.

# Writing XML to a File
Finally, if we want to write the entire Element Tree into a file, we can do something like this:

In [ ]:
filename = '../tmp/output.xml'
with open(filename,'w') as f:
    f.write(ET.tostring(root).decode('utf-8'))